In [8]:
import cv2
import numpy as np
import os

In [9]:

video_folder = "/home/persie/Videos/mokap/"
session_id = os.listdir(video_folder)
video_file = []
for folder in session_id:
    files = os.listdir(os.path.join(video_folder, folder))
    for file in files:
        video_file.append(os.path.join(video_folder, folder, file))


In [6]:
#Initialise Background
# To initialise the background, I'm going to take the median value of the video pixels
def initialise_background(video_path):
    cap = cv2.VideoCapture(video_path)    
    # Read the first frame, but discard as first frame is blank
    ret, first_frame = cap.read()
    if not ret:
        print("Failed to read video")
        #return
        
    # Convert to grayscale
    ret, first_frame = cap.read()
    frame_array = cv2.cvtColor(first_frame, cv2.COLOR_BGR2GRAY)
    
    sample_rate = 50
    fno = 1
    while ret:
        if fno % sample_rate == 0:
            ret, frame = cap.retrieve()
            if not ret:
                break
                    # Convert current frame to grayscale
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            frame_array = np.dstack((frame_array,gray))
            fno += 1
        else:
            # read next frame
            ret = cap.grab()
            fno += 1
  
    print(frame_array.shape)  
    median = np.median(frame_array, axis=2).astype(dtype=np.uint8)
    # mean = np.mean(frame_array, axis=2).astype(dtype=np.uint8)    
    # sigma = np.std(frame_array, axis=2).astype(dtype=np.uint8)

    return median
  

In [1]:
#Background Update
def segment_seeds(video_path, bgs):

    cap = cv2.VideoCapture(video_path)
    # Read the first frame
    ret, first_frame = cap.read()
    if not ret:
        print("Failed to read video")
        return
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Convert current frame to grayscale
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        blur = cv2.GaussianBlur(gray, (7, 7), sigmaX=0, sigmaY=0)
        cv2.imshow('blurred frame', blur)
        fg_mask = bgs.apply(blur, None, 0.0)
        #masked_fg = cv2.inRange(gray, variance)
        cv2.imshow('Frame', frame)
        cv2.imshow('FG Mask', fg_mask)
        #cv2.imshow('masked fg', )

        if cv2.waitKey(30) & 0xFF == ord('q'):
            break


    cap.release()
    cv2.destroyAllWindows()


In [2]:
#Background Update
def segment_seeds_2(video_path, bgs_short, bgs_long):

    cap = cv2.VideoCapture(video_path)
    # Read the first frame
    ret, first_frame = cap.read()
    if not ret:
        print("Failed to read video")
        return
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Convert current frame to grayscale
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        fg_mask = bgs_short.apply(gray, None, 0.0)
        move_mask = bgs_long.apply(gray, None, 0.01)
        
        cv2.imshow('Frame', frame)
        cv2.imshow('Short Mask', fg_mask)
        cv2.imshow('Long Mask', move_mask)

        if cv2.waitKey(30) & 0xFF == ord('q'):
            break


    cap.release()
    cv2.destroyAllWindows()

In [3]:
def create_bgs_short(clearest_frame):
    back_sub = cv2.createBackgroundSubtractorMOG2(history=100, varThreshold=9, detectShadows=True)
    
    back_sub.setShadowThreshold(0.49)
    
    back_sub.apply(clearest_frame, None, 1)
  
    return back_sub
    

In [114]:
def change_var_callback(val):
    bgs_long.setVarThreshold(val)
    
def change_hist_callback(val):
    bgs_long.setHistory(val)
    
def change_shadow_callback(val):
    new_val = val / 100
    bgs_long.setShadowThreshold(new_val)

def change_complexity_reduction_callback(val):
    new_val = val / 10000
    bgs_long.setComplexityReductionThreshold(new_val)

def change_bgd_ratio_callback(val):
    new_val = val / 100
    bgs_long.setBackgroundRatio(new_val)
    
    
    
source_window = 'Long Mask'
cv2.namedWindow(source_window, cv2.WINDOW_NORMAL)
max_var = 250
max_hist = 10000
max_shadow = 100
max_complexity = 100

thresh = 10 # initial threshold
cv2.createTrackbar('varThreshold:', source_window, thresh, max_var, change_var_callback)
cv2.createTrackbar('shadow:', source_window, 6, max_shadow, change_shadow_callback)
cv2.createTrackbar('complexityReduction:', source_window, 0, max_complexity, change_complexity_reduction_callback)


segment_seeds_2(video_file[33], bgs_short, bgs_long)

In [4]:
  
def create_bgs_long(video_path, sample_rate=50):

    cap = cv2.VideoCapture(video_path)    
    # Read the first frame, but discard as first frame is blank
    ret, first_frame = cap.read()
    if not ret:
        print("Failed to read video")
        return
    
    back_sub = cv2.createBackgroundSubtractorMOG2(history=1000, varThreshold=11, detectShadows=True)
    back_sub.setShadowThreshold(0.049)
    
    fno = 1
    while ret:
        if fno % sample_rate == 0:
            ret, frame = cap.retrieve()
            if not ret:
                break
            # Convert current frame to grayscale
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            back_sub.apply(gray, None, 0.1)
            fno += 1
        else:
            # read next frame
            ret = cap.grab()
            fno += 1


    return back_sub
  

In [14]:
cap = cv2.VideoCapture("/home/persie/Videos/mokap/240809-1240/240809-1240_cam2_banana_session23.mp4")
cap.set(cv2.CAP_PROP_POS_FRAMES, 10)
ret, frame = cap.read()
clear_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)



In [10]:
median_frame = initialise_background("/home/persie/Videos/mokap/240809-1240/240809-1240_cam2_banana_session23.mp4")

(1080, 1440, 130)


In [13]:
while True:
    cv2.imshow('median', median_frame)
    if cv2.waitKey(30) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [23]:

bgs_short = create_bgs_short(clear_frame)

In [24]:
bgs_long = create_bgs_long("/home/persie/Videos/mokap/240809-1240/240809-1240_cam2_banana_session23.mp4", 15)


In [25]:
segment_seeds_2("/home/persie/Videos/mokap/240809-1240/240809-1240_cam2_banana_session23.mp4", bgs_short, bgs_long)

In [27]:
def thresh_callback(val):
    global thresh
    thresh = val
    

# Create Window
source_window = 'Source'
cv2.namedWindow(source_window, cv2.WINDOW_NORMAL)
max_thresh = 255

thresh = 255 # initial threshold
cv2.createTrackbar('Canny thresh:', source_window, thresh, max_thresh, thresh_callback)
thresh_callback(thresh)


cap = cv2.VideoCapture(video_file[33])
# Read the first frame
ret, first_frame = cap.read()
bgs_short = create_bgs_short(clear_frame)
bgs_long = create_bgs_long(video_file[33])


while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Convert current frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    fg_mask = bgs_short.apply(gray, None, 0)
    move_mask = bgs_long.apply(gray, None, 0.1)

    cv2.imshow('Source', gray)
    cv2.imshow('FG Mask', fg_mask)
    cv2.imshow('Move Mask', move_mask)
    blur_fg = cv2.blur(fg_mask, (3,3))
    # Detect edges using Canny
    canny_output = cv2.Canny(blur_fg, thresh, thresh * 2)
    # Find contours
    contours,_ = cv2.findContours(canny_output, cv2.RETR_TREE, cv2.CHAIN_APPROX_TC89_L1)#cv2.CHAIN_APPROX_SIMPLE)
    # Find the convex hull object for each contour
    hull_list = []
    for i in range(len(contours)):
        hull = cv2.convexHull(contours[i])
        hull_list.append(hull)
    # Draw contours + hull results
    drawing = np.zeros((canny_output.shape[0], canny_output.shape[1], 3), dtype=np.uint8)
    for i in range(len(contours)):
        color = (254 ,254,254)
        cv2.drawContours(drawing, contours, i, color)
        cv2.drawContours(drawing, hull_list, i, color)
    # Show in a window
    cv2.imshow('Contours', drawing)
    


    if cv2.waitKey(30) & 0xFF == ord('q'):
        break


cap.release()
cv2.destroyAllWindows()





In [16]:
bgs = create_bgs_short(clear_frame)

In [21]:
def outlineseed(video_path, bgs):
    cap = cv2.VideoCapture(video_path)
    
    # Read the first frame
    ret, first_frame = cap.read()
    if not ret:
        print("Failed to read video")
        return
    
    kernel = np.ones((3,3),np.uint8)
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Convert current frame to grayscale
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        
        fg_mask = bgs.apply(gray, None, 0.0)
        cv2.imshow('fg_mask', fg_mask)
        
        
        # noise removal
        sure_fg = cv2.threshold(fg_mask, 254, 255, cv2.THRESH_BINARY)[1]
        sure_fg = cv2.erode(sure_fg, kernel, iterations=1)
        

        sure_bg = cv2.threshold(fg_mask, 0, 255, cv2.THRESH_BINARY)[1]
        sure_bg = cv2.erode(sure_bg, kernel, iterations=1)
        sure_bg = cv2.dilate(sure_bg, kernel, iterations=1)
        

        cv2.imshow('sure_fg', sure_fg)
        cv2.imshow('sure_bg', sure_bg)
        unknown = cv2.subtract(sure_bg,sure_fg)
        cv2.imshow('unknown', unknown)
        # Marker labelling
        ret, markers = cv2.connectedComponents(sure_fg)
         
        # Add one to all labels so that sure background is not 0, but 1
        markers = markers+1
        # Now, mark the region of unknown with zero
        markers[unknown==255] = 0
        markers = cv2.watershed(frame,markers)
        frame[markers == -1] = [255,0,0]
        
        cv2.imshow('frame', frame)

        if cv2.waitKey(30) & 0xFF == ord('q'):
            break

        # Update previous frame
        prev_gray = gray

    cap.release()
    cv2.destroyAllWindows()


In [19]:
def show_frame(video_path, frame_id):
    cap = cv2.VideoCapture(video_path)    
    cap.set(cv2.CAP_PROP_POS_FRAMES, frame_id)
    ret, frame = cap.read()
    if not ret:
        print("Failed to read video")
        return
    
    while True:
        cv2.imshow('Frame', frame)
        if cv2.waitKey(30) & 0xFF == ord('q'):
            break


    cap.release()
    cv2.destroyAllWindows()


In [20]:
show_frame("/home/persie/Videos/mokap/240809-1240/240809-1240_cam2_banana_session23.mp4", 10)